In [ ]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [19]:
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, month, year,monotonically_increasing_id,row_number,concat, udf
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType,TimestampNTZType,StringType
from pyspark.sql.window import Window
from pyspark.conf import SparkConf
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime, date
import os
from dotenv import load_dotenv, dotenv_values
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
from binance.exceptions import BinanceRequestException, BinanceAPIException
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import time
import logging
import psycopg2
load_dotenv()

True

In [2]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [3]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .config("spark.jars", "/Users/hamza/Desktop/projects/postgresql-42.7.5.jar") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

25/04/24 17:45:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/24 17:45:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
#read data for 1 crypto
#schema check/validation need to do this, create spark df using schema struct
#implement transformations
#add unique id for crypto,price,time
#split into 3 tables
#upload to postgres db

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #set to debug to capture all levels
if logger.hasHandlers():
    logger.handlers.clear()
logger.propagate = False

In [ ]:
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [5]:
cryptos = ['BTCUSDT','ETHUSDT','LTCUSDT','BNBUSDT','DOGEUSDT']
client_minio = Minio(
        "localhost:9000",  # Make sure you're using port 9000 for the S3 API
        #minio_url,
        access_key = MINIO_USER,
        secret_key = MINIO_PASSWORD,
        secure=False  # Disable SSL if you're not using SSL certificates
        )

In [6]:
objects = client_minio.list_objects("binancedata", prefix="BNBUSDT", recursive=True)
filenames = [obj.object_name for obj in objects]
filenames = [f for f in filenames if "_SUCCESS" not in f]

In [7]:
df_bnb = spark.read.parquet("s3a://binancedata/BNBUSDT/Monthly/year=2019/month=10/part-00001-12233d6e-b55f-4e5a-8285-808d25f4bf53.c000.snappy.parquet")
for file in filenames:
    df = spark.read.parquet(f"s3a://binancedata/{file}")
    df_bnb = df_bnb.union(df)
                            

25/04/24 17:46:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [ ]:
def data_cleaning():
    pass
    #drop duplicates
    #convert data types
    #drop null values

In [ ]:
def df_to_table():
    pass\
    #read from postgres
    #convert table to df and join with dataframe
    #check if id
    #convert dataframe to sql tables

In [ ]:
crypto_dict = [{"name":"Bitcoin","ticker":"BTC"},
               {"name":"Ethereum","ticker":"ETH"},
               {"name":"Litecoin","ticker":"LTC"},
               {"name":"Dogecoin","ticker":"DOGE"},
               {"name":"BNB","ticker":"BNB"}
              ]
crypto_df = spark.createDataFrame(crypto_dict)
#crypto_df = crypto_df.withColumn('id',monotonically_increasing_id())

In [4]:
'''crypto_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/crypto") \
    .option("dbtable", "crypto") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append")\
    .save()'''

'crypto_df.write     .format("jdbc")     .option("url", "jdbc:postgresql://localhost:5432/crypto")     .option("dbtable", "crypto")     .option("user", "postgres")     .option("password", "postgres")     .option("driver", "org.postgresql.Driver")     .mode("append")    .save()'

In [8]:
read_sql = "SELECT * FROM Crypto"

df_crypto = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/crypto") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("query", read_sql)\
    .option("driver", "org.postgresql.Driver")\
    .load()

In [9]:
df_crypto = df_crypto.withColumn("trading pair", concat(df_crypto.ticker, lit("USDT")))
df_crypto = df_crypto.filter(col("trading pair") == "BNBUSDT")
crypto_id = df_crypto.collect()[0]['id']
df_bnb = df_bnb.withColumn("crypto_id",lit(crypto_id)) 

In [16]:
df_bnb = df_bnb.withColumn("time_id",int())

In [18]:
df_bnb.show(5)

+-------------------+-----------+-----------+------------------+---------+------------+
|           datetime| Open Price|Close Price|            Volume|crypto_id|     time_id|
+-------------------+-----------+-----------+------------------+---------+------------+
|2019-10-01 01:00:00|15.84930000|19.90990000| 53481014.91000000|        5|           0|
|2019-01-01 00:00:00| 6.11390000| 6.22000000| 72126339.77000000|        5|  8589934592|
|2019-10-01 01:00:00|15.84930000|19.90990000| 53481014.91000000|        5| 17179869184|
|2019-11-01 00:00:00|19.90870000|15.71180000| 46441747.10000000|        5| 25769803776|
|2019-12-01 00:00:00|15.70300000|13.71610000| 38189582.55000000|        5| 34359738368|
|2019-02-01 00:00:00| 6.22500000|10.27790000|110685041.86000000|        5| 42949672960|
|2019-03-01 00:00:00|10.27770000|17.53760000|115379333.44000000|        5| 51539607552|
|2019-04-01 01:00:00|17.52010000|22.03910000| 83676534.62000000|        5| 60129542144|
|2019-05-01 01:00:00|22.03000000

In [29]:
def generate_time_id(dt_value):
    ts = dt_value.strftime("%Y%m")
    return int(ts)

In [30]:
dt_udf = udf(generate_time_id,IntegerType())

In [43]:
df_with_udf = df_bnb.withColumn("time_id", dt_udf(df_bnb["datetime"]))


In [44]:
df_with_udf.show(5)

+-------------------+-----------+-----------+-----------------+---------+-------+
|           datetime| Open Price|Close Price|           Volume|crypto_id|time_id|
+-------------------+-----------+-----------+-----------------+---------+-------+
|2019-10-01 01:00:00|15.84930000|19.90990000|53481014.91000000|        5| 201910|
|2019-01-01 00:00:00| 6.11390000| 6.22000000|72126339.77000000|        5| 201901|
|2019-10-01 01:00:00|15.84930000|19.90990000|53481014.91000000|        5| 201910|
|2019-11-01 00:00:00|19.90870000|15.71180000|46441747.10000000|        5| 201911|
|2019-12-01 00:00:00|15.70300000|13.71610000|38189582.55000000|        5| 201912|
+-------------------+-----------+-----------+-----------------+---------+-------+
only showing top 5 rows



In [45]:
df_with_udf = df_with_udf.withColumn("year", year(df_with_udf["datetime"]))

In [46]:
df_with_udf = df_with_udf.withColumn("month", month(df_with_udf["datetime"]))

In [47]:
df_time = df_with_udf.select(['crypto_id','time_id','year','month'])

In [53]:
df_time = df_time.withColumnRenamed('time_id','id')

In [56]:
df_time.show(5)

+---------+------+----+-----+
|crypto_id|    id|year|month|
+---------+------+----+-----+
|        5|201910|2019|   10|
|        5|201901|2019|    1|
|        5|201910|2019|   10|
|        5|201911|2019|   11|
|        5|201912|2019|   12|
+---------+------+----+-----+
only showing top 5 rows



In [54]:
df_time.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/crypto") \
    .option("dbtable", "Time") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append")\
    .save()

25/04/24 18:54:17 ERROR Executor: Exception in task 0.0 in stage 109.0 (TID 552)
java.sql.BatchUpdateException: Batch entry 0 INSERT INTO Time ("crypto_id","id","year","month") VALUES (('5'::int4),('201910'::int4),('2019'::int4),('10'::int4)) was aborted: ERROR: duplicate key value violates unique constraint "time_pkey"
  Detail: Key (id)=(201910) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:52)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2421)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:580)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(PgStatement.java:889)
	at org.postgresql.jdbc.PgStatement.executeBatch(PgStatement.java:913)
	at org.postgresql.jdbc.PgPreparedStatement.executeBatch(PgPreparedStatement.java:17

Py4JJavaError: An error occurred while calling o511.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 109.0 failed 1 times, most recent failure: Lost task 0.0 in stage 109.0 (TID 552) (hamzas-laptop executor driver): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO Time ("crypto_id","id","year","month") VALUES (('5'::int4),('201910'::int4),('2019'::int4),('10'::int4)) was aborted: ERROR: duplicate key value violates unique constraint "time_pkey"
  Detail: Key (id)=(201910) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:52)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2421)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:580)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(

In [ ]:
#redefine your crypto tables
#apply functional programming
#